#### ....coming from the lab-sql-9-ainara-guerra sql document

In [6]:
#but first, all kinds of libraries 
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass  # To get the password without showing the input

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#sns.set_theme(style="whitegrid")

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression #logistic reggression is for classification
from sklearn.metrics import accuracy_score

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:


In [7]:
password = getpass("INSERT YOUR F*KING PASSWORD: ")

In [8]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [9]:
data_may = pd.read_sql_query('SELECT * FROM sakila.rentals_may', engine)
data_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [10]:
data_june = pd.read_sql_query('SELECT * FROM sakila.rentals_june', engine)
data_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


## Check the number of rentals for each customer for May.

In [11]:
query_may = "SELECT customer_id, count(rental_id) as total_rentals_may_per_customer FROM rentals_may GROUP BY customer_id ORDER BY total_rentals_may_per_customer DESC;"
count_may = pd.read_sql_query(query_may, engine)
count_may.head()

,customer_id,total_rentals_may_per_customer
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6


## Check the number of rentals for each customer for June
#### Hint: You can store the results from the two queries in two separate dataframes

In [12]:
query_june = "SELECT customer_id, count(rental_id) as total_rentals_june_per_customer FROM rentals_june GROUP BY customer_id ORDER BY total_rentals_june_per_customer DESC;"
count_june = pd.read_sql_query(query_june, engine)
count_june.head()

,customer_id,total_rentals_june_per_customer
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

In [13]:
rental_merge_inner = count_may.merge(count_june, on="customer_id", how ="inner")
rental_merge_inner

,customer_id,total_rentals_may_per_customer,total_rentals_june_per_customer
0,197,8,8
1,506,7,5
2,109,7,5
3,269,6,3
4,239,6,5
...,...,...,...
507,234,1,8
508,431,1,7
509,351,1,3
510,10,1,5


In [14]:
rental_merge_inner["compar_may_june"]="Decrease in rental"



In [15]:
rental_merge_inner["compar_may_june"][rental_merge_inner['total_rentals_may_per_customer']<rental_merge_inner['total_rentals_june_per_customer']] = "Increase in rental"
rental_merge_inner["compar_may_june"][rental_merge_inner['total_rentals_may_per_customer']==rental_merge_inner['total_rentals_june_per_customer']] = "Equal"

rental_merge_inner.head()

C:\Users\narag\AppData\Local\Temp\ipykernel_39796\3268850003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_merge_inner["compar_may_june"][rental_merge_inner['total_rentals_may_per_customer']<rental_merge_inner['total_rentals_june_per_customer']] = "Increase in rental"
C:\Users\narag\AppData\Local\Temp\ipykernel_39796\3268850003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_merge_inner["compar_may_june"][rental_merge_inner['total_rentals_may_per_customer']==rental_merge_inner['total_rentals_june_per_customer']] = "Equal"


,customer_id,total_rentals_may_per_customer,total_rentals_june_per_customer,compar_may_june
0,197,8,8,Equal
1,506,7,5,Decrease in rental
2,109,7,5,Decrease in rental
3,269,6,3,Decrease in rental
4,239,6,5,Decrease in rental


In [16]:
final_df = rental_merge_inner.copy()
final_df = final_df.set_index('customer_id')
final_df.head()

,total_rentals_may_per_customer,total_rentals_june_per_customer,compar_may_june
customer_id,,,
197,8,8,Equal
506,7,5,Decrease in rental
109,7,5,Decrease in rental
269,6,3,Decrease in rental
239,6,5,Decrease in rental


In [19]:
def checking7(final_df, customer_id):
    result = final_df["compar_may_june"][customer_id]

    if result == "Decrease in rental":
        print ("Decrease in rentals")
    if result == "Increase in rental":
        print ("Increase in rentals")
    else:
        print ("Equal")

In [21]:
checking7(final_df, 109)

Decrease in rentals
Equal


In [22]:
checking7(final_df, 259)

Increase in rentals
